# AFL Model - Part 2 - Feature Creation

These tutorials will walk you through how to construct your own basic AFL model. The output will be odds for each team to win, which will be shown on [The Hub](https://www.betfair.com.au/hub/tools/models/afl-prediction-model/).

In this notebook we will walk you through creating features from our dataset, which was cleaned in the first tutorial. Feature engineering is an integral part of the Data Science process. Creative and smart features can be the difference between an average performing model and a model profitable which beats the market odds.


## Grabbing Our Dataset
First, we will import our required modules, as well as the prepare_afl_data function which we created in our afl_data_cleaning script. This essentially cleans all the data for us so that we're ready to explore the data and make some features.

In [849]:
# Import modules
from afl_data_cleaning import prepare_afl_data
import pandas as pd
pd.set_option('display.max_columns', None)
from afl_data_cleaning import prepare_match_results
import warnings
warnings.filterwarnings('ignore')

In [852]:
# Use the prepare_afl_data function to prepare the data for us; this function condenses what we walked through in the previous tutorial
afl_data = prepare_afl_data()

In [856]:
# Look at the last 4 rows - this should be the last 2 games played
afl_data.tail(4)

,Team,odds,Date,match_details,home_team,away_team,Behinds,Game,Goals,Home?,Margin,Opposition,Opposition Behinds,Opposition Goals,Opposition Points,Points,Round,Venue,Season,Status,GA,CP,UP,ED,CM,MI5,One.Percenters,BO,K,HB,D,M,G,B,T,HO,I50,CL,CG,R50,FF,FA,AF,SC,CCL,SCL,SI,MG,TO,ITC,T5
2946,North Melbourne,1.3182,2018-06-23,Western Bulldogs v North Melbourne,Western Bulldogs,North Melbourne,5,15316,12,0,2,Western Bulldogs,9,11,75,77,14,Docklands,2018,Away,9,149,208,264,20,10,48,8,189,163,352,81,12,5,53,43,52,31,62,39,23,23,1422,1600,16.0,15.0,71.0,5341.0,83.0,82.0,8.0
2947,Western Bulldogs,4.1127,2018-06-23,Western Bulldogs v North Melbourne,Western Bulldogs,North Melbourne,9,15316,11,1,-2,North Melbourne,5,12,77,75,14,Docklands,2018,Home,7,144,249,295,13,10,42,9,207,178,385,96,11,9,53,22,55,27,55,39,23,23,1528,1700,9.0,18.0,92.0,6015.0,82.0,85.0,20.0
2948,Carlton,14.7519,2018-06-24,Collingwood v Carlton,Collingwood,Carlton,5,15317,9,0,-20,Collingwood,13,11,79,59,14,M.C.G.,2018,Away,6,112,228,255,15,11,45,5,207,129,336,124,9,3,58,14,37,27,57,39,19,30,1483,1536,13.0,14.0,71.0,4970.0,70.0,61.0,8.0
2949,Collingwood,1.0734,2018-06-24,Collingwood v Carlton,Collingwood,Carlton,13,15317,11,1,20,Carlton,5,9,59,79,14,M.C.G.,2018,Home,9,125,262,303,17,14,48,8,219,168,387,108,11,9,51,40,53,31,49,27,30,19,1609,1764,9.0,22.0,94.0,5487.0,61.0,71.0,8.0


## What Each Column Refers To
Below is a DataFrame which outlines what each column refers to.

In [857]:
column_abbreviations = pd.read_csv("data/afl_data_columns_mapping.csv")
column_abbreviations

,Feature Abbreviated,Feature
0,GA,Goal Assists
1,CP,Contested Possessions
2,UP,Uncontested Possessions
3,ED,Effective Disposals
4,CM,Contested Marks
5,MI5,Marks Inside 50
6,One.Percenters,One Percenters
7,BO,Bounces
8,K,Kicks
9,HB,Handballs


## Feature Creation
Now let's think about what features we can create. We have a enormous amount of stats to sift through. To start, let's create some simple features based on our domain knowledge of Aussie Rules.


### Creating Efficiency Features
#### Disposal Efficiency
Disposal efficiency is pivotal in Aussie Rules football. If you are dispose of the ball effectively you are much more likely to score and much less likely to concede goals than if you dispose of it ineffectively.

Let's create a disposal efficiency feature by dividing Effective Disposals by Disposals.

#### Inside 50/Rebound 50 Efficiency
Similarly, one could hypothesise that teams who keep the footy in their Inside 50 regularly will be more likely to score, whilst teams who are effective at getting the ball out of their Defensive 50 will be less likely to concede. Let's use this logic to create Inside 50 Efficiency and Rebound 50 Efficiency features.

The formula used will be:
```
Inside 50 Efficiency = R50_Opponents / I50 (lower is better).
Rebound 50 Efficiency = R50 / I50_Opponents (higher is better).
```

To create these features we will need the opposition's Inside 50s/Rebound 50s. So we will split out data into two DataFrames, create a new DataFrame by joining these two DataFrames on the Game, calculate our efficiency features, then join our features with our main afl_data DataFrame.

In [846]:
# Create Disposal Efficiency feature - the proportion of disposals which are 'effective disposals'
afl_data['disposal_efficiency'] = afl_data['ED'] / afl_data['D']

# Create Rebound 50 Efficiency feature - the proportion of Opposition Inside 50s which are rebounded

# First we will define a function which creates a new DataFrame with Opposition Statistics on the same row as the Team Statistics 
def get_opp_stats_df(df):
    # Filter the DataFrames by whether it was a home or away game
    home_df = df[df['Status'] == 'Home']
    away_df = df[df['Status'] == 'Away']

    # Rename the away columns so we know that they are from the away team
    home_df_renamed = home_df.rename(columns={col: col + '_Opp' for col in home_df.columns if col != 'Game'})
    away_df_renamed = away_df.rename(columns={col: col + '_Opp' for col in home_df.columns if col != 'Game'})

    # Merge the two DataFrames on the Game
    merged_1 = pd.merge(home_df, away_df_renamed, on=['Game'])
    merged_2 = pd.merge(away_df, home_df_renamed, on=['Game'])

    # Append the DataFrames together and then sort by the Game Id, reset the index and drop unrequired columns
    merged = merged_1.append(merged_2).sort_values(by='Game').reset_index(drop=True)
    return merged

opponent_stats_df = get_opp_stats_df(afl_data)

# Create Rebound 50 Efficiency - the proportion of Rebound 50s from opposition Inside 50s
opponent_stats_df['R50_efficiency'] = opponent_stats_df['R50'] / opponent_stats_df['I50_Opp']

# Create Inside 50 Efficiency - the proportion of opposition Rebound 50s from Inside 50s
opponent_stats_df['I50_efficiency'] = opponent_stats_df['R50_Opp'] / opponent_stats_df['I50']

# Merge features to main afl_data DataFrame
afl_data = pd.merge(afl_data, opponent_stats_df[['Team', 'Game', 'R50_efficiency', 'I50_efficiency']], on=['Team', 'Game'])

### Creating Rolling Averages as Features
Next, we will create rolling averages of statistics such as Tackles, which we will use as features.

It is fair to assume that a team's performance in a certain stat may have predictive power to the overall result. And in general, if a team consistently performs well in this stat, this may have predictive power to the result of their future games. We can't simply train a model on stats from the game which we are trying to predict (i.e. data that we don't have before the game begins), as this will leak the result. We need to train our model on past data. One way of doing this is to train our model on average stats over a certain amount of games. If a team is averaging high in this stat, this may give insight into if they are a strong team. Similarly, if the team is averaging poorly in this stat (relative to the team they are playing), this may have predictive power and give rise to a predicted loss.

To do this we will create a function which calculates the rolling averages, known as create_rolling_averages, which takes our cleaned DataFrame as an input, as well as the window to calculate the average over, and the columns to create the average for and output a new DataFrame with the rolling averages instead of the statistics.

In [777]:
# Define a function which returns a DataFrame with the rolling averages for each game. Cols refers to the columns which we want
# to create a rolling average for
def create_rolling_averages(df, window, cols):
    new_cols = [col + '_ave_{}'.format(window) for col in cols]
    df[new_cols] = df.groupby('Team')[cols].apply(lambda x: x.rolling(window).mean().shift())
    df = df.drop(columns=cols)
    return df

In [847]:
# Get index of GA - as this is the first column where we will be taking the average
cols_indx_start = afl_data.columns.get_loc("GA")

afl_avgs = create_rolling_averages(afl_data, 6, afl_data.columns[cols_indx_start:])

As you can see our function worked perfectly! Note that we are creating a rolling average and then shifting this average down a row to ensure that we use previous data for the average so we don't leak the result. As a result, the first window rows have NaN values as these rows are used to calculate the rolling average. For example, if you are calculating a rolling average with a window of 2, with the values \[3, 5, 8\], then the associated rolling average would be \[NaN, 4, 6.5\]. When we shift this down (to ensure no data leakage), we get \[NaN, NaN, 4\].

### Creating an Elo Feature
Another feature which we could create is an Elo feature. If you don't know what Elo is, go ahead and read our article on it [here](https://www.betfair.com.au/hub/better-betting/betting-strategies/tennis/tennis-elo-modelling/). We have also written a guide on using elo to model the 2018 FIFA World Cup [here](https://www.betfair.com.au/hub/how-to-use-elo-to-model-the-world-cup/).

Essentially, Elo ratings increase if you win. The amount the rating increases is based on how strong the opponent is relative to the team who won. Weak teams get more points for beating stronger teams than they do for beating weaker teams, and vice versa for losses (teams lose points for losses).

Mathematically, Elo ratings can also assign a probability for winning or losing based on the two Elo Ratings of the teams playing.

So let's get into it. We will first define a function which calculates the elo for each team and applies these elos to our DataFrame.

In [842]:
# Define a function which finds the elo for each team in each game and returns a dictionary with the game ID as a key and the
# elos as the key's value, in a list. It also outputs the probabilities and a dictionary of the final elos for each team
def elo_applier(df, k_factor):
    # Initialise a dictionary with default elos for each team
    elo_dict = {team: 1500 for team in df['Team'].unique()}
    elos, elo_probs = {}, {}
    
    # Sort by Game and then only grab the Home Games so that the same isn't repeated
    df = df.sort_values(by=['Game']).reset_index(drop=True)
    df = df[df['Home?'] == 1]
    df = df.drop_duplicates(subset='Game', keep='first')
    
    # Loop over the rows in the DataFrame
    for index, row in df.iterrows():
        # Get the Game ID
        game_id = row['Game']
        
        # If the game already has the elos for the home and away team in the elos dictionary, go to the next game
        if game_id in elos.keys():
            continue
        
        # Get the team and opposition
        home_team = row['home_team']
        away_team = row['away_team']
        
        # Get the team and opposition elo score
        home_team_elo = elo_dict[home_team]
        away_team_elo = elo_dict[away_team]
        
        # Calculated the probability of winning for the team and opposition
        prob_win_home = 1 / (1 + 10**((away_team_elo - home_team_elo) / 400))
        prob_win_away = 1 - prob_win_home
        
        # Add the elos and probabilities our elos dictionary and elo_probs dictionary based on the Game ID
        elos[game_id] = [home_team_elo, away_team_elo]
        elo_probs[game_id] = [prob_win_home, prob_win_away]
        
        # Calculate the new elos of each team
        if row['Margin'] > 0: # Team wins; update both teams' elo
            new_home_team_elo = home_team_elo + k_factor*(1 - prob_win_home)
            new_away_team_elo = away_team_elo + k_factor*(0 - prob_win_away)
        elif row['Margin'] < 0: # Away team wins; update both teams' elo
            new_home_team_elo = home_team_elo + k_factor*(0 - prob_win_home)
            new_away_team_elo = away_team_elo + k_factor*(1 - prob_win_away)
        elif row['Margin'] == 0: # Drawn game' update both teams' elo
            new_home_team_elo = home_team_elo + k_factor*(0.5 - prob_win_home)
            new_away_team_elo = away_team_elo + k_factor*(0.5 - prob_win_away)
        
        # Update elos in elo dictionary
        elo_dict[home_team] = new_home_team_elo
        elo_dict[away_team] = new_away_team_elo
    
    return elos, elo_probs, elo_dict

In [848]:
# Grab the elos and elo probabilities for each game then map them to our DataFrame
elos, probs, elo_dict = elo_applier(afl_avgs, 24)
afl_avgs['home_elo'] = afl_avgs['Game'].map(elos).apply(lambda x: x[0])
afl_avgs['away_elo'] = afl_avgs['Game'].map(elos).apply(lambda x: x[1])

In [672]:
afl_avgs[['home_team', 'away_team', 'Margin', 'home_elo', 'away_elo']].tail()

,home_team,away_team,Margin,home_elo,away_elo
2933,Gold Coast,St Kilda,-2,1297.533962,1391.872570
2934,Hawthorn,Adelaide,-56,1590.973387,1590.027977
2935,Hawthorn,Adelaide,56,1590.973387,1590.027977
2936,Geelong,Richmond,18,1682.657509,1594.703066
2937,Geelong,Richmond,-18,1682.657509,1594.703066


Great! now we have both rolling averages for stats as a feature, and the elo of the teams! Let's have a quick look at the current elo standings with a k-factor of 24, out of curiosities sake.

In [673]:
for team in sorted(elo_dict, key=elo_dict.get)[::-1]:
    print(team, elo_dict[team])

Sydney 1693.7343582142216
Geelong 1667.6829439063786
West Coast 1635.679285063372
Richmond 1609.6776310945465
Hawthorn 1602.94073405867
Port Adelaide 1579.6938118646788
Adelaide 1578.060630111068
GWS 1565.3212773608266
Collingwood 1523.1536167317115
Melbourne 1504.0298883964674
Essendon 1460.952034987102
North Melbourne 1460.4458414834025
Western Bulldogs 1458.5393767249375
Fremantle 1449.4838638325766
St Kilda 1400.6920137594548
Gold Coast 1288.714518498949
Carlton 1285.0240092382885
Brisbane 1236.1741646733478


This looks extremely similar to the currently AFL ladder, so this is a good sign for elo being an effective predictor of winning.

### Creating an Elo Adjusted Average Margin Feature
Now that we have elo defined, we can adjust the margin for each game based on the elo for each team, and then average this margin, to create a new feature. To do this we will first restructure our elo columns so that they are associated with the actual team on the row, rather than the 'home' and 'away' elo measures we currently have.

In [674]:
# Create a function which maps the current home and away team elos to 'Team Elo' and 'Opposition Elo'
def map_elos(df):
    home_df = df[df['Home?'] == 1]
    away_df = df[df['Home?'] == 0]
    home_df['elo'] = home_df['home_elo']
    home_df['elo_Opp'] = home_df['away_elo']
    away_df['elo'] = away_df['away_elo']
    away_df['elo_Opp'] = away_df['home_elo']
    final_df = home_df.append(away_df).sort_values(by=['Game', 'Home?'])
    return final_df

In [675]:
# Map the elos using our function
afl_avgs = map_elos(afl_avgs)

# Create Adjusted Margin and then Average it over a 6 game window
afl_avgs['Adj_elo_ave_margin'] = afl_avgs['Margin'] * afl_avgs['elo_Opp'] / afl_avgs['elo']
afl_avgs = create_rolling_averages(afl_avgs, 6, ['Adj_elo_ave_margin'])

### Creating a 'Form Between the Teams' Feature
It is well known in Aussie Rules that often some teams perform better against certain teams than others. If we isolate our features to pure stats based on previous games not between the teams playing, or elo ratings, we won't account for any relationships between certain teams. An example is the [Kennett Curse](https://en.wikipedia.org/wiki/Kennett_curse), where Geelong won 11 consecutive games against Hawthorn, despite being similarly matched teams. Let's create a feature which calculates how many games a team has won against their opposition over a given window of games.

To do this, we will need to use historical data that dates back well before our current DataFrame starts at. Otherwise we will be using a lot of our games to calculate form, meaning we will have to drop these rows before feeding it into an algorithm. So let's use our prepare_match_results function which we defined in the afl_data_cleaning tutorial to grab a clean DataFrame of all match results since 1897. We can then calculate the form and join this to our current DataFrame.

In [676]:
match_results = prepare_match_results("data/afl_match_results.csv")
# Filter for games after 2004
match_results = match_results.iloc[25000:].reset_index(drop=True)

In [677]:
match_results.head(3)

,Behinds,Date,Game,Goals,Home?,Margin,Opposition,Opposition Behinds,Opposition Goals,Opposition Points,Points,Round,Team,Venue
0,8,2004-04-24,12501,14,1,49,West Coast,7,6,43,92,5,Carlton,Princes Park
1,7,2004-04-24,12501,6,0,-49,Carlton,8,14,92,43,5,West Coast,Princes Park
2,10,2004-04-24,12502,20,1,65,North Melbourne,17,8,65,130,5,St Kilda,Docklands


In [678]:
# Define a function which calculates the form between teams over a given window
def form_between_teams(df, window):
    num_wins_over_opposition = []
    # Iterate over rows
    for idx, row in df.iterrows():
        # Get a DataFrame of recent games between the teams
        recent_games_between_teams = df[(df['Team'] == row['Team']) & (df['Opposition'] == row['Opposition'])].loc[:idx-1][-window:]
        # Calculate the number of wins for the current team
        num_wins = recent_games_between_teams[recent_games_between_teams['Margin'] > 0].shape[0]
        # Append this to a list
        num_wins_over_opposition.append(num_wins)
    # Add the new feature as a column
    df['form_over_opposition_{}'.format(window)] = num_wins_over_opposition
    return df

In [679]:
# Find the historical form between teams and create a new DataFrame with the form_over_opposition feature in it
form = form_between_teams(match_results, 5)[['Date', 'Team', 'Opposition', 'form_over_opposition_5']]

# Join the new DataFrame to our main DataFrame
afl_avgs = pd.merge(afl_avgs, form, on=['Date', 'Team', 'Opposition'])

### Creating an Average Elo of Beaten Opposition Feature
Whilst the average margin between the teams feature allows us to capture form between specific teams, and average statistics allows us to capture general form of a team in certain areas, we currently cannot capture how teams perform against better teams and how teams perform against worse teams. For example, in 2018 Essendon have beaten teams such as West Coast and Geelong (top teams), but lost to poor teams like Carlton and Western Bulldogs. Let's create this feature and then inspect Essendon's Average Elo of Beaten teams to check if it's relatively higher than other teams.

So let's create this feature. While we're at it, we'll also create an 'average elo of opponent' for losses, too. This will allow us to capture the opposite; if teams perform worse against poor teams.

To do this we will create a function which loops over the DataFrame and appends the elo of opponents into a list within a dictionary based on the team name if they win/lose, depending on if we want the 'beaten' feature or 'lost' feature. Let's calculate both for a window of 6.

In [680]:
def create_ave_elo_opponent(df, window, beaten_or_lost='beaten'):
    elos_of_opponents = {team: [] for team in df['Team'].unique()}
    ave_elo_opponents = []
    
    # Loop over rows of the DataFrame
    for idx, row in df.iterrows():
        # Grab the mean elos of opponents beaten and append it to a list
        if len(elos_of_opponents[row['Team']]) >= window:
            ave_elo_opponents.append(np.mean(elos_of_opponents[row['Team']][-window:]))
        else:
            ave_elo_opponents.append(np.nan)
        
        if beaten_or_lost == 'beaten':
            # Update the elos of opponents beaten for this game (if the team wins, add their opponents elo to the dictionary)
            if row['Margin'] > 0 and row['Home?'] == 1:
                elos_of_opponents[row['Team']].append(row['away_elo']) 
            elif row['Margin'] > 0 and row['Home?'] == 0:
                elos_of_opponents[row['Team']].append(row['home_elo'])
        
        elif beaten_or_lost == 'lost':
            # Update the elos of opponents lost to for this game (if the team wins, add their opponents elo to the dictionary)
            if row['Margin'] < 0 and row['Home?'] == 1:
                elos_of_opponents[row['Team']].append(row['away_elo']) 
            elif row['Margin'] < 0 and row['Home?'] == 0:
                elos_of_opponents[row['Team']].append(row['home_elo'])
                
    df['average_elo_opponents_{}_{}'.format(beaten_or_lost, window)] = ave_elo_opponents
    return df

In [681]:
afl_avgs = create_ave_elo_opponent(afl_avgs, 6, beaten_or_lost='beaten')
afl_avgs = create_ave_elo_opponent(afl_avgs, 6, beaten_or_lost='lost')

In [682]:
# Get Essendon's most recent game as of 29/06/2018
afl_avgs.iloc[2920:2925]

,Team,odds,Date,match_details,home_team,away_team,Behinds,Game,Goals,Home?,Margin,Opposition,Opposition Behinds,Opposition Goals,Opposition Points,Points,Round,Venue,Season,Status,GA_ave_6,CP_ave_6,UP_ave_6,ED_ave_6,CM_ave_6,MI5_ave_6,One.Percenters_ave_6,BO_ave_6,K_ave_6,HB_ave_6,D_ave_6,M_ave_6,G_ave_6,B_ave_6,T_ave_6,HO_ave_6,I50_ave_6,CL_ave_6,CG_ave_6,R50_ave_6,FF_ave_6,FA_ave_6,AF_ave_6,SC_ave_6,CCL_ave_6,SCL_ave_6,SI_ave_6,MG_ave_6,TO_ave_6,ITC_ave_6,T5_ave_6,disposal_efficiency_ave_6,R50_efficiency_ave_6,I50_efficiency_ave_6,home_elo,away_elo,elo,elo_Opp,Adj_elo_ave_margin_ave_6,form_over_opposition_5,average_elo_opponents_beaten_6,average_elo_opponents_lost_6
2920,Essendon,1.7522,2018-06-10,Brisbane v Essendon,Brisbane,Essendon,12,15303,12,0,22,Brisbane,14,8,62,84,12,Gabba,2018,Away,8.000000,149.000000,222.000000,263.000000,11.666667,11.000000,48.500000,8.333333,214.166667,160.666667,374.833333,87.333333,10.333333,9.166667,75.666667,44.833333,51.000000,38.833333,58.833333,38.0,19.166667,22.000000,1597.666667,1647.333333,9.833333,29.000000,90.000000,5606.500000,71.500000,71.000000,13.666667,0.699826,0.703122,0.699194,1241.596213,1455.529987,1455.529987,1241.596213,-12.082570,3,1539.251720,1486.833099
2921,Brisbane,2.2998,2018-06-10,Brisbane v Essendon,Brisbane,Essendon,14,15303,8,1,-22,Essendon,12,12,84,62,12,Gabba,2018,Home,9.166667,139.166667,221.666667,268.000000,10.166667,12.666667,58.833333,3.666667,209.166667,152.500000,361.666667,91.833333,12.500000,8.166667,65.166667,48.666667,50.666667,36.833333,54.333333,37.5,21.666667,19.833333,1562.666667,1590.000000,13.333333,23.500000,95.333333,5357.333333,68.500000,69.000000,8.833333,0.740019,0.682925,0.680963,1241.596213,1455.529987,1241.596213,1455.529987,-13.837659,2,1420.809115,1503.709431
2922,Adelaide,1.2967,2018-06-10,Fremantle v Adelaide,Fremantle,Adelaide,14,15304,9,0,-3,Fremantle,11,10,71,68,12,Perth Stadium,2018,Away,8.000000,159.833333,211.333333,258.833333,10.000000,9.333333,46.500000,5.833333,223.000000,148.833333,371.833333,85.000000,12.833333,9.000000,71.500000,40.000000,50.166667,39.166667,49.000000,34.5,20.833333,16.666667,1604.500000,1632.166667,11.333333,27.833333,87.500000,5870.166667,74.166667,75.333333,11.166667,0.691727,0.711205,0.703276,1424.570652,1607.829228,1607.829228,1424.570652,2.216107,3,1468.535840,1524.610822
2923,Fremantle,4.2942,2018-06-10,Fremantle v Adelaide,Fremantle,Adelaide,11,15304,10,1,3,Adelaide,14,9,68,71,12,Perth Stadium,2018,Home,7.000000,138.500000,236.500000,266.166667,10.833333,9.333333,49.000000,2.500000,199.000000,164.500000,363.500000,94.000000,9.333333,7.166667,60.833333,41.833333,45.833333,37.166667,56.666667,38.0,18.333333,20.500000,1513.166667,1566.166667,13.333333,23.833333,76.500000,4939.166667,73.333333,66.333333,8.333333,0.731912,0.661502,0.727621,1424.570652,1607.829228,1424.570652,1607.829228,-37.244469,2,1420.186158,1566.086969
2924,Collingwood,2.6448,2018-06-11,Melbourne v Collingwood,Melbourne,Collingwood,13,15305,20,0,42,Melbourne,7,14,91,133,12,M.C.G.,2018,Away,10.666667,146.166667,286.333333,322.666667,12.000000,12.000000,49.333333,7.500000,228.666667,200.833333,429.500000,110.833333,13.833333,7.833333,65.500000,48.500000,55.666667,36.833333,55.666667,38.0,21.666667,18.333333,1788.166667,1756.000000,12.500000,24.333333,112.166667,5793.833333,71.833333,70.000000,11.000000,0.750092,0.731493,0.692177,1516.210791,1510.972714,1510.972714,1516.210791,9.398294,1,1449.666623,1598.556956


As we can see, Essendon has an average elo of opponents beaten of 1539 for a window of 6, whereas Collingwood (who have been a solid performing team recently) only have a rating of 1449. This means that Essendon's recent wins are against 'good' teams. Similarly Essendon's average elo of opponents that they lose to is 1486, whereas Collingwood's is 1598.

In [683]:
# Create the home_win column - this will be our target variable
afl['home_win'] = afl.apply(lambda x: 1 if x['Margin'] > 0 else 0, axis=1)

# Create regular margin rolling average
afl_avgs = create_rolling_averages(afl_avgs, 6, ['Margin'])

## Wrapping it Up
We now have a fairly decent amount of features. Some other features which could be added include whether the game is in a major Capital city outisde of Mebourne (i.e. Sydney, Adelaide or Peth), how many 'Elite' players are playing (which could be judged by average SuperCoach scores over 110, for example), as well as your own metrics for attacking and defending.